In [2]:
pip install geopandas 

   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   --------------------------- ------------ 8.7/12.6 MB 42.9 MB/s eta 0:00:01
   ---------------------------------------  12.6/12.6 MB 46.9 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 27.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ----------------------------------- ---- 10.2/11.5 MB 50.7 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.5 MB 51.6 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 25.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/16.2 MB ? eta -:--:--
   ---------------- ----------------------- 6.6/16.2 MB 34.6 MB/s eta 0:00:01
   --------------------------------- ------ 13.6/16.2 MB 34.3 MB/s eta 0:00:01
   ---------------------------------------  16.0/16.2 MB 34.5 MB/s eta 0:00:01
   ---------------------------------------- 16.2/16.2 MB 22.3 MB/s eta 0:00:00
  

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import geopandas as gpd
from shapely.ops import unary_union

In [3]:
# Load the GeoJSON file
communes = gpd.read_file('data/communes4326.geojson')

In [6]:
# Group by 'CANTON' and merge the polygons
cantons = communes.dissolve(by='CANTON', aggfunc='sum')

In [7]:
# Save the result to a new GeoJSON file
cantons.to_file('data/cantons.geojson', driver='GeoJSON')

In [4]:
# Group by 'DISTRICT' and merge the polygons
districts = communes.dissolve(by='DISTRICT', aggfunc='sum')

In [5]:
# Save the result to a new GeoJSON file
districts.to_file('data/district.geojson', driver='GeoJSON')

In [8]:
# Merge all the polygons in communes to create a single polygon for the country
country = gpd.GeoDataFrame(geometry=[unary_union(communes.geometry)], crs=communes.crs)

In [9]:
# Save the result to a new GeoJSON file
country.to_file('lux.geojson', driver='GeoJSON')